In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons, make_circles, make_classification, make_multilabel_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score

from sklearn_ubu.disturbing_neighbors import DisturbingNeighbors
from sklearn_ubu.random_oracles import RandomOracles
from sklearn_ubu.rotation_forest import RotationForest

In [13]:
seed=0
num_datasets=5
num_features=5
num_classes=2
num_labels=1
list_tuples=[]
names_rows=[]
for i in range(num_datasets):
    num_features=num_features+3
    num_classes=num_classes+2
    num_labels=num_labels+1
    list_tuples.append((num_features,num_classes,num_labels))
    names_rows.append('dataset' + str(i+1) )
names_rows.append('mean')

In [14]:
X=[]
y=[]
y_test=[]
for i in range(num_datasets):
    X.append(make_multilabel_classification(
        n_samples=2500, n_features=list_tuples[i][0], n_classes=list_tuples[i][1],
        n_labels=list_tuples[i][2],  random_state=seed)[0])
    y.append(make_multilabel_classification(
        n_samples=2500, n_features=list_tuples[i][0], n_classes=list_tuples[i][1],
        n_labels=list_tuples[i][2],  random_state=seed)[1])

In [15]:
classifier=[]
classifier.append(DisturbingNeighbors(n_neighbors=20, n_features=2))
classifier.append(RandomOracles(n_oracles=5))
classifier.append(RotationForest(n_groups=5))
classifier.append(DecisionTreeClassifier())

In [16]:
cross_v=5
ensembles_accuracy=[]
for i in range(len(classifier)):
    accuracy=[]
    for j in range(num_datasets):
        accuracy.append(np.mean(cross_val_score(classifier[i], X[j], y[j],scoring='accuracy', cv=cross_v)))
    accuracy.append(np.mean(accuracy))
    ensembles_accuracy.append(accuracy)

In [17]:
ensembles_accuracy = np.array(ensembles_accuracy).T
ensembles_accuracy=list(ensembles_accuracy)

ensembles_accuracy

[array([ 0.37  ,  0.4224,  0.4372,  0.3608]),
 array([ 0.138 ,  0.1864,  0.1852,  0.1332]),
 array([ 0.0516,  0.0564,  0.0532,  0.046 ]),
 array([ 0.008 ,  0.0096,  0.0084,  0.0056]),
 array([ 0.0036,  0.0024,  0.002 ,  0.0036]),
 array([ 0.11424,  0.13544,  0.1372 ,  0.10984])]

A test with 5 different data sets has been realized and the average of the 5 has been calculated. Thoses sets have been analyzed with the 3 algorithms realized and the DecisionTreeClassifier. The accuracy measure has been used, with which we obtain the precision.

Below is a table to see the results:

In [18]:
names_columns=['Disturbing Neighbors', 'Random Oracles', 'Rotation Forest', 'Decision Tree Classifier']
df = pd.DataFrame(ensembles_accuracy, \
       columns=(names_columns), \
       index=(names_rows)) 
df

,Disturbing Neighbors,Random Oracles,Rotation Forest,Decision Tree Classifier
dataset1,0.37000,0.42240,0.4372,0.36080
dataset2,0.13800,0.18640,0.1852,0.13320
dataset3,0.05160,0.05640,0.0532,0.04600
dataset4,0.00800,0.00960,0.0084,0.00560
dataset5,0.00360,0.00240,0.0020,0.00360
mean,0.11424,0.13544,0.1372,0.10984
